In [1]:
import numpy as np
import tensorflow as tf
import os

import sionna
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, ITURadioMaterial, PathSolver

import mitsuba as mi
import drjit as dr

%matplotlib
import matplotlib.pyplot as plt

# mi.set_variant('scalar_rgb')

2025-04-09 01:08:50.351581: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744175330.377075   73515 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744175330.385361   73515 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744175330.411931   73515 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744175330.411960   73515 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744175330.411963   73515 computation_placer.cc:177] computation placer alr

Using matplotlib backend: module://matplotlib_inline.backend_inline


In [2]:
min = (0,0, 0)
max = (500, 500, 1000)

spacing = 25
num_tx = 1

scene_dir = "minis"
scene_files = [f for f in os.listdir(scene_dir) if f.endswith('.xml')]
print(scene_files)

['row5_col5.xml', 'row2_col6.xml', 'row1_col6.xml', 'row2_col2.xml', 'row6_col0.xml', 'row0_col3.xml', 'row1_col1.xml', 'row4_col1.xml', 'row2_col3.xml', 'row1_col4.xml', 'row0_col7.xml', 'row4_col2.xml', 'row1_col7.xml', 'row5_col6.xml', 'row0_col5.xml', 'row3_col0.xml', 'row3_col3.xml', 'row1_col3.xml', 'row5_col4.xml', 'row7_col5.xml', 'row6_col1.xml', 'row3_col7.xml', 'row0_col4.xml', 'row3_col1.xml', 'row6_col2.xml', 'row4_col7.xml', 'row5_col3.xml', 'row7_col6.xml', 'row5_col0.xml', 'row2_col4.xml', 'row0_col6.xml', 'row2_col1.xml', 'row1_col5.xml', 'row7_col2.xml', 'row5_col1.xml', 'row7_col3.xml', 'row7_col1.xml', 'row2_col0.xml', 'row6_col5.xml', 'row3_col2.xml', 'row4_col0.xml', 'row1_col2.xml', 'row6_col7.xml', 'row5_col2.xml', 'row4_col4.xml', 'row6_col6.xml', 'row2_col7.xml', 'row5_col7.xml', 'row7_col4.xml', 'row3_col6.xml', 'row6_col4.xml', 'row2_col5.xml', 'row6_col3.xml', 'row7_col0.xml', 'row3_col5.xml', 'row7_col7.xml', 'row0_col2.xml', 'row3_col4.xml', 'row4_col3.xm

In [3]:
scene_file = scene_files[1]
scene = load_scene(os.path.join(scene_dir, scene_file))
output_dir = os.path.splitext(scene_file)[0]

try:
    os.mkdir(output_dir)
except OSError:
    print('dir already exists')

dir already exists


In [4]:
rx_points = []

x_start, y_start, _ = min
x_end, y_end, _ = max
xx = np.linspace(x_start, x_end, spacing)
yy = np.linspace(y_start, y_end, spacing)
ray_pos = np.array([(x, y, 1000) for x in xx for y in yy])

z_rays = mi.Ray3f(o=ray_pos.T, d=(0, 0, -1))
z_hits = scene.mi_scene.ray_intersect(z_rays)

for z_hit, valid in zip(z_hits.p.numpy().T, z_hits.is_valid()):
    if (not valid):
        continue
    # start = int(np.ceil(z_hit[2] / spacing) * spacing)
    start = int(z_hit[2] + 10)
    
    # print(start)
    for i in range(start, 200, spacing):
        # print([z_hit[0], z_hit[1], i])
        rx_points.append(np.array([z_hit[0], z_hit[1], i]))
    # rx_points.append(z_hit)
# print(rx_points)

In [5]:
## Include points that did not have a valid hitpoint due to lack of floor right now
for i, v in enumerate(z_hits.is_valid()):
    if(not v):
        r = ray_pos[i]
        for i in range(0, 200, spacing):
            rx_points.append(np.array([r[0], r[1], i]))
        # rx_points.append(np.array([r[0], r[1], 0]))

In [6]:
scene.frequency # Defaults to 3.5GHz

mat = ITURadioMaterial(name='concrete', itu_type='concrete', thickness=0.1, scattering_coefficient=0.0, xpd_coefficient=0.0)

# Set all objects to concrete for now
for obj in scene.objects.values():
    obj.radio_material = mat

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=1,
                          horizontal_spacing=1,
                          polarization="V",
                          pattern="iso")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=0.5,
                          horizontal_spacing=0.5,
                          polarization="V",
                          pattern="iso")

# Create transmitter
# tx = Transmitter(name="tx",
#               position=[200,200,100],
#               orientation=[0,0,0])
# scene.add(tx)

In [7]:
# Create receivers
for rx_i in range(len(rx_points)):
#     print(rx_points[rx_i])
    rx = Receiver(name=f"rx_{rx_i}",
            position=rx_points[rx_i],
            orientation=[0,0,0])
    scene.add(rx)

In [8]:
# tx_pos = []
# for tx_i in range(num_tx):
    
points = np.array(rx_points)
filtered = points[(points[:,2] <= 50)]
# print(filtered)
filtered[:,2] += 5
# print(filtered)

np.random.seed(42)
c = np.random.choice(filtered.shape[0], size=num_tx, replace=False)
print(c)

tx_pos = filtered[c]

for tx_i in range(num_tx):
    tx = Transmitter(name=f"tx_{tx_i}",
            position=tx_pos[tx_i],
            orientation=[0,0,0])
    scene.add(tx)

[654]


In [9]:
p_solver  = PathSolver()
paths = p_solver(scene=scene,
                 max_depth=5,
                 los=False,
                 specular_reflection=True,
                 diffuse_reflection=False,
                 refraction=False,
                 synthetic_array=False,
                 seed=41)

In [10]:
# import cProfile
# p_solver  = PathSolver()

# cProfile.run(p_solver(scene=scene,
#                  max_depth=5,
#                  los=False,
#                  specular_reflection=True,
#                  diffuse_reflection=False,
#                  refraction=False,
#                  synthetic_array=False,
#                  seed=41))


In [11]:
scene.preview(paths=paths)

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, …

Export fields to csv's

transmitter locs
receiver locs
a
primitives
pickle paths obj


In [12]:
def log_flat_arr(file, X):
    shape = X.shape
    flattened = X.reshape(shape[0], -1)
    np.savetxt(fname=os.path.join(output_dir, file), X=flattened, header=f'{shape}')


In [13]:
# scene.transmitters.
tx_log = paths.sources.numpy().T
log_flat_arr("tx_locations", tx_log)
# np.savetxt(fname=os.path.join(output_dir, "tx_locations"), X=tx_log)

rx_log = paths.targets.numpy().T
log_flat_arr("rx_locations", rx_log)

# np.savetxt(fname=os.path.join(output_dir, "rx_locations"), X=rx_log)

real_a_log = paths.a[0].numpy()
# print(real_a_log.shape)
imaginary_a_log = paths.a[1].numpy()
# print(real_a_log.reshape(real_a_log.shape[0], -1).shape)
# real_a_log = real_a_log.reshape(real_a_log.shape[0], -1)
# imaginary_a_log = imaginary_a_log.reshape(imaginary_a_log.shape[0], -1)
log_flat_arr("real_a", real_a_log)
log_flat_arr("imaginary_a", imaginary_a_log)



# np.savetxt(fname=os.path.join(output_dir, "real_a"), X=real_a_log)
# np.savetxt(fname=os.path.join(output_dir, "imaginary_a"), X=imaginary_a_log)

# paths.valid.numpy().shape

# paths.a
# paths.valid.shape
valid_log = paths.valid.numpy()

# valid_shape = valid_log.shape
# valid_log = valid_log.reshape(valid_log.shape[0], -1)

# np.savetxt(fname=os.path.join(output_dir, "valid"), X=valid_log, header=f'{valid_shape}')
log_flat_arr("valid", valid_log)

primitives_log = paths.primitives.numpy()
log_flat_arr("primitives", primitives_log)

vertices_log = paths.primitives.numpy()
log_flat_arr("vertices", vertices_log)



In [14]:
type(valid_log)

numpy.ndarray

In [ ]:
# read_in = np.genfromtxt(fname=os.path.join(output_dir, "real_a"))
# np.array_equal(read_in.reshape(4842, 1, 2, 1, 8), paths.a[0].numpy())

ValueError: cannot reshape array of size 19368 into shape (4842,1,2,1,8)